In [ ]:
# Objetivo: limpar uma base de dados imobiliários da Inglaterra

In [145]:
# 1 - Importar pacotes

import pandas as pd
import numpy as np

In [146]:
# 2 - Pegar a base de dados

df = pd.read_csv('https://raw.githubusercontent.com/OpenClassrooms-Student-Center/Supervised-Learning/master/cleaning.csv')

In [147]:
df

,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,12/04/1999,1,"£729,000",apartment
1,Ashford,05/08/2017,unknown,"£699,000",semi-detached
2,Stratford-on-Avon,29/03/2012,3,NaN,detached
3,Canterbury,01/07/2009,2,"£529,000",teraced
4,Camden,16/12/2001,1,"£616,000",apartment
5,Rugby,01/03/2003,-,"£247,000",detached
6,Hampstead,05/03/2016,2,£0,terraced
7,Clapham,05/07/2001,363,"£543,000",apartment
8,Stratford-on-Avon,10th May 2010,3,"£420,000",detached
9,Camden,16/12/2001,1,"£616,000",apartment


In [148]:
# 3 - Ver os data types

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   location            10 non-null     object
 1   date_of_sale        10 non-null     object
 2   number_of_bedrooms  10 non-null     object
 3   price               9 non-null      object
 4   type                10 non-null     object
dtypes: object(5)
memory usage: 528.0+ bytes


In [149]:
# 4 - Remover linhas duplicadas

    # Pois é improvável que duas casas sejam vendidas no mesmo dia, na mesma locação e no mesmo preço

df.drop_duplicates()

,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,12/04/1999,1,"£729,000",apartment
1,Ashford,05/08/2017,unknown,"£699,000",semi-detached
2,Stratford-on-Avon,29/03/2012,3,NaN,detached
3,Canterbury,01/07/2009,2,"£529,000",teraced
4,Camden,16/12/2001,1,"£616,000",apartment
5,Rugby,01/03/2003,-,"£247,000",detached
6,Hampstead,05/03/2016,2,£0,terraced
7,Clapham,05/07/2001,363,"£543,000",apartment
8,Stratford-on-Avon,10th May 2010,3,"£420,000",detached


In [150]:
# 5 - Coluna type: corrigir a palavra terrace

df.type = df.type.replace('teraced','terrace')
df.type = df.type.replace('terraced','terrace')
df.type.unique()

array(['apartment', 'semi-detached', 'detached', 'terrace'], dtype=object)

In [151]:
# 6 - Coluna number_of_bedrooms: trocar valores não numéricos para nan

    # Obs.: como a coluna era objeto (string), usei series.string.isnumeric()

not_bedroom_number = df[~df['number_of_bedrooms'].str.isnumeric()] # É um data frame que traz o resultado diferente (~) da função str.isnumeric()
not_bedroom_number = not_bedroom_number['number_of_bedrooms'].unique() # Pegar os valores únicos desse data frame

df['number_of_bedrooms'] = df['number_of_bedrooms'].replace(not_bedroom_number, np.nan) # Substituir os valores not_bedroom_number por np.nan

In [152]:
# 7 - Coluna number_of_bedrooms: transformar a coluna em int, para poder filtrar os outliers (casas com mais de 30 quartos) e depois fazer a mean imputation

    # Obs.: If ‘coerce’, then invalid parsing will be set as NaN.

df['number_of_bedrooms'] = pd.to_numeric(df['number_of_bedrooms'], errors='coerce')
df['number_of_bedrooms'] = df['number_of_bedrooms'].astype('Int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   location            10 non-null     object
 1   date_of_sale        10 non-null     object
 2   number_of_bedrooms  8 non-null      Int64 
 3   price               9 non-null      object
 4   type                10 non-null     object
dtypes: Int64(1), object(4)
memory usage: 538.0+ bytes


In [153]:
# 8 -  Coluna number_of_bedrooms: trocar os outliers (casas com mais de 30 quartos) para nan

    # Pegar a linha com o número de quartos maior que 30 (outliers)
outlier_bedrooms = df[df['number_of_bedrooms'] > 30]['number_of_bedrooms'].unique() # Filtrar um data frame só com mais de 30 quartos, e pegar a coluna number_of_bedrooms
                                                                                    # Colocar o unique, porque vira um array (sem o unique é uma series, e o replace não funciona)

    # Substituir o outlier por NaN, para futuramente virar uma mean imput
df['number_of_bedrooms'] = df['number_of_bedrooms'].replace(outlier_bedrooms,np.nan)

In [154]:
df

,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,12/04/1999,1,"£729,000",apartment
1,Ashford,05/08/2017,<NA>,"£699,000",semi-detached
2,Stratford-on-Avon,29/03/2012,3,NaN,detached
3,Canterbury,01/07/2009,2,"£529,000",terrace
4,Camden,16/12/2001,1,"£616,000",apartment
5,Rugby,01/03/2003,<NA>,"£247,000",detached
6,Hampstead,05/03/2016,2,£0,terrace
7,Clapham,05/07/2001,<NA>,"£543,000",apartment
8,Stratford-on-Avon,10th May 2010,3,"£420,000",detached
9,Camden,16/12/2001,1,"£616,000",apartment


In [155]:
# 9 - Coluna number_of_bedrooms: pegar a média

    # Pegar o data frame sem number_of_bedrooms nulos ou maiores que 30 (outliers)
df1 = df[~((df['number_of_bedrooms'].isna())|(df['number_of_bedrooms']>30))]

    # Pegar a média do number_of_bedrooms e arredondar (pois número de quartos é um valor inteiro)
mean_bedrooms = df1.number_of_bedrooms.mean()
mean_bedrooms = int(round(mean_bedrooms,0))
mean_bedrooms

2

In [156]:
# 10 - Coluna number_of_bedrooms: fazer o mean imputation dos NaN

df['number_of_bedrooms'] = df['number_of_bedrooms'].replace(np.nan, mean_bedrooms)
df

,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,12/04/1999,1,"£729,000",apartment
1,Ashford,05/08/2017,2,"£699,000",semi-detached
2,Stratford-on-Avon,29/03/2012,3,NaN,detached
3,Canterbury,01/07/2009,2,"£529,000",terrace
4,Camden,16/12/2001,1,"£616,000",apartment
5,Rugby,01/03/2003,2,"£247,000",detached
6,Hampstead,05/03/2016,2,£0,terrace
7,Clapham,05/07/2001,2,"£543,000",apartment
8,Stratford-on-Avon,10th May 2010,3,"£420,000",detached
9,Camden,16/12/2001,1,"£616,000",apartment


In [157]:
# 11 - Coluna price: transformar o outlier (preço 0) em NaN, para futuramente fazer o mean imputation

df['price'] = df['price'].replace('£0',np.nan)
df['price']

0    £729,000
1    £699,000
2         NaN
3    £529,000
4    £616,000
5    £247,000
6         NaN
7    £543,000
8    £420,000
9    £616,000
Name: price, dtype: object

In [158]:
# 12 - Coluna price: trocar o NaN para um texto, para as etapas futuras
    
df['price'] = df['price'].fillna('fazer mean imput')

In [159]:
# 13 - Coluna price: replace ',' por ''; porque o python entende que a vírgula é um separador de números (e não um separador de milhares)
    
df['price'] = df['price'].apply(lambda x: x.replace(',',''))

In [160]:
# 14 - Coluna price: remover o primeiro caractere

df['price'] = df['price'].apply(lambda x: x[1:])

In [161]:
# 15 - Coluna price: transformar a coluna em float, para depois fazer a mean imputation

    # Obs.: If ‘coerce’, then invalid parsing will be set as NaN.

df['price'] = pd.to_numeric(df['price'], errors='coerce').astype('Float64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   location            10 non-null     object 
 1   date_of_sale        10 non-null     object 
 2   number_of_bedrooms  10 non-null     Int64  
 3   price               8 non-null      Float64
 4   type                10 non-null     object 
dtypes: Float64(1), Int64(1), object(3)
memory usage: 548.0+ bytes


In [162]:
# 16 - Coluna price: calcular a média

    # Pegar um data frame sem valores de price nulos
df2 = df[~df['price'].isna()]

    # Pegar a média de price
mean_price = df2['price'].mean()
mean_price

549875.0

In [163]:
# 17 - Coluna price: substituir NaN com mean_price

df['price'] = df['price'].replace(np.nan, mean_price)

In [164]:
df

,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,12/04/1999,1,729000.0,apartment
1,Ashford,05/08/2017,2,699000.0,semi-detached
2,Stratford-on-Avon,29/03/2012,3,549875.0,detached
3,Canterbury,01/07/2009,2,529000.0,terrace
4,Camden,16/12/2001,1,616000.0,apartment
5,Rugby,01/03/2003,2,247000.0,detached
6,Hampstead,05/03/2016,2,549875.0,terrace
7,Clapham,05/07/2001,2,543000.0,apartment
8,Stratford-on-Avon,10th May 2010,3,420000.0,detached
9,Camden,16/12/2001,1,616000.0,apartment


In [165]:
# 18 - Coluna price: renomear coluna

df = df.rename(columns = {'price':'price_euros'})

In [167]:
# 19 - Coluna date_of_sale: formatar a coluna

df['date_of_sale'] = pd.to_datetime(df['date_of_sale'], errors = 'coerce')

In [168]:
# 20 - Data Frame formatado

df

,location,date_of_sale,number_of_bedrooms,price_euros,type
0,Clapham,1999-12-04,1,729000.0,apartment
1,Ashford,2017-05-08,2,699000.0,semi-detached
2,Stratford-on-Avon,2012-03-29,3,549875.0,detached
3,Canterbury,2009-01-07,2,529000.0,terrace
4,Camden,2001-12-16,1,616000.0,apartment
5,Rugby,2003-01-03,2,247000.0,detached
6,Hampstead,2016-05-03,2,549875.0,terrace
7,Clapham,2001-05-07,2,543000.0,apartment
8,Stratford-on-Avon,2010-05-10,3,420000.0,detached
9,Camden,2001-12-16,1,616000.0,apartment
